# Back Propagation Example 1

In [2]:
import numpy as np
from utils import *
from stratified import *
from neuralnetwork import *

### Forward Propagate

In [3]:
# Theta 1
# 0.40000  0.10000  
# 0.30000  0.20000  
theta1 = np.array([[0.4, 0.1],[0.3,0.2]])
# Theta 2 
#   0.70000  0.50000  0.60000
theta2 = np.array([0.7,0.5,0.6])
weightlist1= [theta1,theta2]

In [4]:
# Training set
# 	Training instance 1
# 		x: [0.13000]
# 		y: [0.90000]
# 	Training instance 2
# 		x: [0.42000]
# 		y: [0.23000]
# Training instance 1
trainingcategory = {'x1':'numerical', 'y':'class_numerical'}
trainingdata1 = np.array([0.13,0.9])
trainingdata2 = np.array([0.42,0.23])
inputdata1 = np.append(1,trainingdata1[0])
inputdata2 = np.append(1,trainingdata2[0])
exceptout1 = trainingdata1[1]
exceptout2 = trainingdata2[1]
lambda1 = 0

In [5]:
def forwardtest(inputdata,weightl,expectedout):
    current_layer_a = inputdata
    print('current_a at 1 is',current_layer_a)
    current_layer_index = 0
    alist = []
    alist.append(current_layer_a)
    for theta in weightl:
        z = np.dot(theta,current_layer_a)
        a = g(z)
        current_layer_a = np.append(1,a) if (current_layer_index+1 != len(weightl)) else a
        print('current_a at',current_layer_index+2,'is',current_layer_a)
        alist.append(current_layer_a)
        current_layer_index += 1
    result = current_layer_a
    print('prediction is', result)
    print('exceptout is', expectedout)
    print('cost is', costfunction(expectedout,result))
    return result, costfunction(expectedout,result), alist

In [6]:
r1,j1,a1 = forwardtest(inputdata1,weightlist1,exceptout1)
# Computing the error/cost, J, of the network
# 	Processing training instance 1
# 	Forward propagating the input [0.13000]
# 		a1: [1.00000   0.13000]

# 		z2: [0.41300   0.32600]
# 		a2: [1.00000   0.60181   0.58079]

# 		z3: [1.34937]
# 		a3: [0.79403]

# 		f(x): [0.79403]
# 	Predicted output for instance 1: [0.79403]
# 	Expected output for instance 1: [0.90000]
# 	Cost, J, associated with instance 1: 0.366

current_a at 1 is [1.   0.13]
current_a at 2 is [1.        0.601807  0.5807858]
current_a at 3 is 0.7940274264318581
prediction is 0.7940274264318581
exceptout is 0.9
cost is 0.36557477431084995


In [7]:
r2,j2,a2 = forwardtest(inputdata2,weightlist1,exceptout2)
	# Processing training instance 2
	# Forward propagating the input [0.42000]
	# 	a1: [1.00000   0.42000]

	# 	z2: [0.44200   0.38400]
	# 	a2: [1.00000   0.60874   0.59484]

	# 	z3: [1.36127]
	# 	a3: [0.79597]

	# 	f(x): [0.79597]
	# Predicted output for instance 2: [0.79597]
	# Expected output for instance 2: [0.23000]
	# Cost, J, associated with instance 2: 1.276

current_a at 1 is [1.   0.42]
current_a at 2 is [1.         0.60873549 0.59483749]
current_a at 3 is 0.7959660671522611
prediction is 0.7959660671522611
exceptout is 0.23
cost is 1.2763768066887786


In [8]:
jlist1 = np.array([j1,j2])
numberofinstance1 = 2

In [9]:
def overallcost(jlist,n,weightl):
    s = sumofweights(weightl,bias=1)*lambda1/(2*n)
    jsum = np.sum(jlist)
    return jsum/n + s

In [10]:
overallcost(jlist1,numberofinstance1,weightlist1)

0.8209757904998143

### Back Propagate

In [12]:
def blame(predict_output, expected_output, weights_list, a_list, biasterm=True): # This is to find out the delta function
    deltalist = []
    delta_layer_l = predict_output - expected_output
    deltalist.append(delta_layer_l)
    i = len(weights_list)-1
    current_delta = delta_layer_l

    while i > 0:
        attributeblame = np.dot(a_list[i+1],(1-a_list[i+1]))
        rawblame = np.dot(weights_list[i].T,current_delta)
        delta_layer_now = np.dot(rawblame,attributeblame)
        if biasterm:
            delta_layer_now[0] = 1 # the first attribute is the bias
            current_delta = delta_layer_now[1:] # the first attribute is the bias
        else:
            current_delta = delta_layer_now
        deltalist.append(current_delta)
        i-=1
    deltalist.reverse()
    
    return deltalist

In [13]:
def delta(weightl,alist,expect,actual):
    delta_layer_n = actual-expect
    deltalist = []
    deltalist.append(delta_layer_n)
    i = len(weightl)-1
    current_layer_delta = delta_layer_n

    while i > 0:
        step1 = np.multiply(np.multiply(np.dot(weightl[i].T,current_layer_delta),alist[i]),(1-alist[i]))
        delta_layer_now = step1
        current_delta = delta_layer_now[1:]
        deltalist.append(current_delta)
        i-=1
    deltalist.reverse()
    return deltalist

In [17]:
def gradientD(weights_list,delta_list,a_list,biasterm=True):
    gradlist = []
    for i in range(len(weights_list)):
        anow = a_list[i]
        deltanow = np.array([delta_list[i]]).T
        dotproduct = deltanow*anow
        # print('dotshape',dotproduct.shape)
        gradlist.append(dotproduct)
    return gradlist

In [14]:
delta1_1 = delta(weightlist1,a1,exceptout1,r1)
	# Computing gradients based on training instance 1
	# 	delta3: [-0.10597]
	# 	delta2: [-0.01270   -0.01548]
print(delta1_1)

[array([-0.01269739, -0.01548092]), -0.10597257356814194]


In [19]:
		# Gradients of Theta2 based on training instance 1:
		# 	-0.10597  -0.06378  -0.06155  

		# Gradients of Theta1 based on training instance 1:
		# 	-0.01270  -0.00165  
		# 	-0.01548  -0.00201 
gradd1_1 = gradientD(weightlist1,delta1_1,a1)
print(gradd1_1)

[array([[-0.01269739, -0.00165066],
       [-0.01548092, -0.00201252]]), array([-0.10597257, -0.06377504, -0.06154737])]


In [16]:
delta1_2 = delta(weightlist1,a2,exceptout2,r2)
	# Computing gradients based on training instance 2
	# 	delta3: [0.56597]
	# 	delta2: [0.06740   0.08184]
print(delta1_2)

[array([0.06739994, 0.08184068]), 0.5659660671522612]


In [20]:
		# Gradients of Theta2 based on training instance 2:
		# 	0.56597  0.34452  0.33666  

		# Gradients of Theta1 based on training instance 2:
		# 	0.06740  0.02831  
		# 	0.08184  0.03437  
gradd1_2 = gradientD(weightlist1,delta1_2,a2)
print(gradd1_2)

[array([[0.06739994, 0.02830797],
       [0.08184068, 0.03437309]]), array([0.56596607, 0.34452363, 0.33665784])]


In [ ]:
def transposelistoflist(l):
    newlistoflist = []
    for i in range(len(l[0])):
        newlist = []
        for j in range(len(l)):
            newlist.append(l[j][i])
        newlistoflist.append(newlist)
    return newlistoflist

In [24]:
listofgradient = [gradd1_1,gradd1_2]
gradientP1 = [lambda1*t for t in weightlist1]
grad_D_transpose = transposelistoflist(listofgradient)
grad_D_sum = [np.sum(t,axis=0) for t in grad_D_transpose]
update_gradients = []
for i in range(len(grad_D_sum)):
    update_gradients.append((grad_D_sum[i] + gradientP1[i])*(1/numberofinstance1))


In [26]:
print(update_gradients)
	# The entire training set has been processes. Computing the average (regularized) gradients:
	# 	Final regularized gradients of Theta1:
	# 		0.02735  0.01333  
	# 		0.03318  0.01618  

	# 	Final regularized gradients of Theta2:
	# 		0.23000  0.14037  0.13756  

[array([[0.02735127, 0.01332866],
       [0.03317988, 0.01618028]]), array([0.22999675, 0.1403743 , 0.13755523])]


# Back Propagation Example 2

In [ ]:
# Theta 1
# 0.40000  0.10000  
# 0.30000  0.20000  
theta1 = np.array([[0.4, 0.1],[0.3,0.2]])
# Theta 2 
#   0.70000  0.50000  0.60000
theta2 = np.array([0.7,0.5,0.6])
weightlist1= [theta1,theta2]